In [ ]:
# Healthcare Fraud Detection - Simplified Evaluation Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                           precision_recall_curve, auc, average_precision_score, 
                           precision_score, recall_score, f1_score, roc_curve)
from sklearn.utils.class_weight import compute_class_weight
import joblib

# Create directories
import os

In [ ]:

# 1. Load and Prepare Data
print("📊 Loading data...")
beneficiary_df = pd.read_csv('/content/Train_Beneficiarydata.csv')
inpatient_df = pd.read_csv('/content/Train_Inpatientdata.csv')
outpatient_df = pd.read_csv('/content/Train_Outpatientdata.csv')
labels_df = pd.read_csv('/content/Train_labels.csv')

print(f"Data shapes - Beneficiary: {beneficiary_df.shape}, Inpatient: {inpatient_df.shape}, Outpatient: {outpatient_df.shape}, Labels: {labels_df.shape}")

# 2. Simple Feature Engineering
print("🛠️ Creating features...")

def create_simple_features(inpatient_df, outpatient_df, labels_df):
    features_list = []
    
    for provider in labels_df['Provider'].unique():
        provider_features = {'Provider': provider}
        
        # Get provider claims
        inpatient_claims = inpatient_df[inpatient_df['Provider'] == provider]
        outpatient_claims = outpatient_df[outpatient_df['Provider'] == provider]
        
        # Basic features
        provider_features['Total_Claims'] = len(inpatient_claims) + len(outpatient_claims)
        provider_features['Inpatient_Ratio'] = len(inpatient_claims) / provider_features['Total_Claims'] if provider_features['Total_Claims'] > 0 else 0
        
        # Amount features
        if len(inpatient_claims) > 0:
            provider_features['Avg_Inpatient_Amount'] = inpatient_claims['InscClaimAmtReimbursed'].mean()
            provider_features['Total_Inpatient_Amount'] = inpatient_claims['InscClaimAmtReimbursed'].sum()
        else:
            provider_features['Avg_Inpatient_Amount'] = 0
            provider_features['Total_Inpatient_Amount'] = 0
            
        if len(outpatient_claims) > 0:
            provider_features['Avg_Outpatient_Amount'] = outpatient_claims['InscClaimAmtReimbursed'].mean()
            provider_features['Total_Outpatient_Amount'] = outpatient_claims['InscClaimAmtReimbursed'].sum()
        else:
            provider_features['Avg_Outpatient_Amount'] = 0
            provider_features['Total_Outpatient_Amount'] = 0
        
        provider_features['Total_Amount'] = provider_features['Total_Inpatient_Amount'] + provider_features['Total_Outpatient_Amount']
        
        features_list.append(provider_features)
    
    features_df = pd.DataFrame(features_list)
    final_df = pd.merge(features_df, labels_df, on='Provider', how='left')
    return final_df

# Create features
final_df = create_simple_features(inpatient_df, outpatient_df, labels_df)
print(f"✅ Features created: {final_df.shape}")

# 3. Prepare Modeling Data
final_df['Fraud'] = final_df['PotentialFraud'].map({'Yes': 1, 'No': 0})
feature_cols = [col for col in final_df.columns if col not in ['Provider', 'PotentialFraud', 'Fraud']]
X = final_df[feature_cols].fillna(0)
y = final_df['Fraud']

print(f"🎯 Fraud rate: {y.mean():.3f} ({y.sum()} fraud cases out of {len(y)})")

# 4. Rigorous Validation Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

print(f"📊 Data splits - Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")

# 5. Handle Class Imbalance
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weight))
print(f"⚖️ Class weights: {class_weight_dict}")

# 6. Train Models with Regularization
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        class_weight=class_weight_dict,
        random_state=42,
        max_depth=10,  # Regularization
        min_samples_split=10  # Regularization
    ),
    'Logistic Regression': LogisticRegression(
        class_weight=class_weight_dict,
        random_state=42,
        max_iter=1000,
        C=0.1  # Regularization
    )
}

# Train and validate
print("🚀 Training models...")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    val_f1 = f1_score(y_val, y_val_pred)
    print(f"  {name} - Validation F1: {val_f1:.4f}")

# Select best model
best_model_name = 'Random Forest'  # Based on typical performance
best_model = models[best_model_name]
print(f"🏆 Best model: {best_model_name}")

# 7. Comprehensive Evaluation on Test Set
print("\n" + "="*60)
print("📈 COMPREHENSIVE MODEL EVALUATION")
print("="*60)

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate all required metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

print(f"🎯 PERFORMANCE METRICS:")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print(f"PR-AUC:    {pr_auc:.4f}")

# Classification Report
print(f"\n📋 CLASSIFICATION REPORT:")
print(classification_report(y_test, y_pred, target_names=['Non-Fraud', 'Fraud']))

# 8. Confusion Matrix
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted Non-Fraud', 'Predicted Fraud'],
            yticklabels=['Actual Non-Fraud', 'Actual Fraud'])
plt.title('Confusion Matrix')

# 9. ROC Curve
plt.subplot(1, 3, 2)
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

# 10. Precision-Recall Curve
plt.subplot(1, 3, 3)
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
plt.plot(recall_curve, precision_curve, label=f'PR Curve (AUC = {pr_auc:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()

plt.tight_layout()
plt.show()

# 11. Cost-Based Analysis
print(f"\n💰 COST-BASED ANALYSIS")
print("="*50)

# Realistic cost assumptions
investigation_cost = 10000  # Cost to investigate a provider
avg_fraud_amount = 50000   # Average fraud amount

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

fp_cost = fp * investigation_cost
fn_cost = fn * avg_fraud_amount
tp_benefit = tp * avg_fraud_amount
net_impact = tp_benefit - fp_cost - fn_cost

print(f"False Positives: {fp} × ${investigation_cost:,} = ${fp_cost:,}")
print(f"False Negatives: {fn} × ${avg_fraud_amount:,} = ${fn_cost:,}")
print(f"True Positives:  {tp} × ${avg_fraud_amount:,} = ${tp_benefit:,}")
print(f"Net Impact: ${net_impact:,}")

if net_impact > 0:
    print("✅ POSITIVE BUSINESS IMPACT")
else:
    print("⚠️  NEGATIVE BUSINESS IMPACT")

# 12. Error Analysis
print(f"\n🔍 ERROR ANALYSIS")
print("="*50)

# Create error analysis dataframe
error_df = X_test.copy()
error_df['Actual'] = y_test.values
error_df['Predicted'] = y_pred
error_df['Predicted_Probability'] = y_pred_proba
error_df['Provider'] = final_df.loc[X_test.index, 'Provider'].values

# Identify error types
error_df['Error_Type'] = 'Correct'
error_df.loc[(error_df['Actual'] == 1) & (error_df['Predicted'] == 0), 'Error_Type'] = 'False Negative'
error_df.loc[(error_df['Actual'] == 0) & (error_df['Predicted'] == 1), 'Error_Type'] = 'False Positive'

print("Error distribution:")
print(error_df['Error_Type'].value_counts())

# 13. Case Studies
print(f"\n📝 CASE STUDIES")
print("="*50)

# False Positive Case Studies (Legitimate providers flagged as fraud)
false_positives = error_df[error_df['Error_Type'] == 'False Positive']
print("\n🚨 FALSE POSITIVES (Legitimate providers flagged as fraud):")

if len(false_positives) >= 2:
    for i in range(min(2, len(false_positives))):
        case = false_positives.iloc[i]
        print(f"\nCase {i+1}:")
        print(f"  Provider: {case['Provider']}")
        print(f"  Prediction Probability: {case['Predicted_Probability']:.3f}")
        print(f"  Key Features:")
        print(f"    - Total Claims: {case['Total_Claims']:.0f}")
        print(f"    - Total Amount: ${case['Total_Amount']:,.0f}")
        print(f"    - Inpatient Ratio: {case['Inpatient_Ratio']:.3f}")
        
        # Analysis
        if case['Total_Amount'] > final_df['Total_Amount'].quantile(0.75):
            print(f"  🔍 Analysis: High total amount may have triggered false positive")
        if case['Inpatient_Ratio'] > 0.7:
            print(f"  🔍 Analysis: High inpatient ratio unusual for legitimate providers")

# False Negative Case Studies (Fraudulent providers missed)
false_negatives = error_df[error_df['Error_Type'] == 'False Negative']
print("\n❌ FALSE NEGATIVES (Fraudulent providers missed):")

if len(false_negatives) >= 2:
    for i in range(min(2, len(false_negatives))):
        case = false_negatives.iloc[i]
        print(f"\nCase {i+1}:")
        print(f"  Provider: {case['Provider']}")
        print(f"  Prediction Probability: {case['Predicted_Probability']:.3f}")
        print(f"  Key Features:")
        print(f"    - Total Claims: {case['Total_Claims']:.0f}")
        print(f"    - Total Amount: ${case['Total_Amount']:,.0f}")
        print(f"    - Avg Outpatient Amount: ${case.get('Avg_Outpatient_Amount', 0):,.0f}")
        
        # Analysis
        if case['Total_Claims'] < final_df['Total_Claims'].quantile(0.5):
            print(f"  🔍 Analysis: Low claim volume may have caused model to miss patterns")
        if case.get('Avg_Outpatient_Amount', 0) < final_df['Avg_Outpatient_Amount'].quantile(0.5):
            print(f"  🔍 Analysis: Average amounts may appear normal")

# 14. Feature Importance Analysis
print(f"\n🎯 FEATURE IMPORTANCE ANALYSIS")
print("="*50)

if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Top 10 Most Important Features:")
    for i, row in feature_importance.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")

# 15. Recommendations for Improvement
print(f"\n💡 RECOMMENDATIONS FOR FUTURE ITERATIONS")
print("="*50)

print("1. Feature Engineering Improvements:")
print("   - Add temporal patterns (seasonal billing)")
print("   - Include beneficiary demographic features")
print("   - Add procedure code analysis")
print("   - Create network analysis features")

print("\n2. Model Improvements:")
print("   - Try ensemble methods (XGBoost, LightGBM)")
print("   - Implement cross-validation")
print("   - Add feature selection")
print("   - Try different classification thresholds")

print("\n3. Data Improvements:")
print("   - Collect more fraud examples")
print("   - Add external data sources")
print("   - Improve feature quality")

# 16. Save Model and Results
print(f"\n💾 SAVING RESULTS")
print("="*50)

# Save model with feature names
model_artifacts = {
    'model': best_model,
    'feature_names': feature_cols,
    'performance': {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc
    }
}

joblib.dump(model_artifacts, 'models/fraud_detection_model.pkl')

# Save evaluation results
results = {
    'test_performance': {
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'roc_auc': float(roc_auc),
        'pr_auc': float(pr_auc)
    },
    'business_impact': {
        'false_positives': int(fp),
        'false_negatives': int(fn),
        'net_impact': float(net_impact)
    },
    'error_analysis': error_df[['Provider', 'Actual', 'Predicted', 'Predicted_Probability', 'Error_Type']].to_dict('records')
}

import json
with open('reports/evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Model saved: models/fraud_detection_model.pkl")
print("✅ Results saved: reports/evaluation_results.json")

NameError: name 'joblib' is not defined